In [1]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.learning_curve import validation_curve
import statsmodels.api as sm
import statsmodels.formula.api as sm
from surprise import Reader, Dataset
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from scipy.special import boxcox1p
import statsmodels.api as sm
import math
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error
import sklearn
from sklearn.cluster import KMeans

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [46]:
def call_api(username, scope):
    token = util.prompt_for_user_token(username,scope,client_id='4849430a2b114eae9d8eaf2c1f14f612',client_secret='11b854b009344409861112df56d9cf44',redirect_uri='http://127.0.0.1:8080/callback/q') 
    return token

In [47]:
token = call_api('mrethana', 'user-top-read')
sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opening https://accounts.spotify.com/authorize?client_id=4849430a2b114eae9d8eaf2c1f14f612&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8080%2Fcallback%2Fq&scope=user-top-read in your browser


Enter the URL you were redirected to: http://127.0.0.1:8080/callback/q?code=AQCP8jUlqmJuE-5NtvCbLda1wXxsuBL4sSuwK91sxZ9wo4ObOEU_OLvcohilJsKCMw8MegXPVEqFG2p6eNAZ6oSnqFS5FpIdu9uXOIc-U9zWAslK48SC3jC-9OA-I51zHTm4z5r61sXxFLFeSGtlHawoMjw4zi0Eke9UfDBxFklgijryJvwiXR7NMdkrqTa8S7uUTxf7320RjUfIlcGolqDXx61t8UQ




In [48]:
# token2 = call_api('mrethana', 'playlist-modify-public')
# sp2 = spotipy.Spotify(auth=token2)

In [49]:
def create_top_tracks_dict(period):
    list_tracks =[]
    for offset in range(0,500,49):
        x = sp.current_user_top_tracks(49,offset,period)
        temp = [track for track in x['items']]
        list_tracks.extend(temp)
#     import pdb; pdb.set_trace()
    top_track_dict = [{'album':item['album']['name'], 'album_id':item['album']['id'], 'album_release':item['album']['release_date'],'artist':item['artists'][0]['name'],'track_name':item['name'],'track_id':item['id']} for item in list_tracks]
    for item in list_tracks:
        for dic in top_track_dict:
            if item['id'] == dic['track_id']:
                if len(item['artists']) > 1:
                    dic['feature'] = item['artists'][1]['name']
                else:
                    dic['feature'] = 'No Feature'
    return top_track_dict
# lt = create_top_tracks_dict('long_term')





In [6]:
def regression_evaluation(target, X_list):
    split = train_test_split(X_list, target,random_state=69)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    X = X_train
    target = y_train
     
    X2 = sm.add_constant(X)
    est = sm.OLS(target, X2)
    est2 = est.fit()
    
    y_predicted = est2.fittedvalues
    y_actual = target
    rmse = math.sqrt(mean_squared_error(y_actual, y_predicted))
    residuals = y_actual-y_predicted
    
    plt.scatter(y_predicted,residuals)
    print(est2.summary())
    plt.scatter(y_predicted,residuals)
    plt.show()
    sns.distplot(residuals)
    print('RMSE=' + str(rmse))

In [7]:
def regression(target, X_list):
    split = train_test_split(X_list, target,random_state=69)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    X = X_train
    target = y_train
     
    X2 = sm.add_constant(X)
    est = sm.OLS(target, X2)
    est2 = est.fit()
    return est2.rsquared

In [8]:
def poly_regression(x, y, degree_range, xlim_min, xlim_max, ylim_min, ylim_max, alpha = .75,s = 5,width = 2.5):
    plt.figure(figsize = (10,10))
    plt.ylim(ylim_min, ylim_max)
    plt.xlim(xlim_min,xlim_max)
    
    #split inputs into training and test sets
    split = train_test_split(x,y)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    x = X_train
    y = y_train
    plt.scatter(x, y, s = s, alpha = alpha)
    
        
    for degree in degree_range:
        poly_model = make_pipeline(PolynomialFeatures(degree),
                               LinearRegression())
        poly_model.fit(x[:, np.newaxis], y)
        xfit = np.linspace(xlim_min,xlim_max, 10000)
        yfit = poly_model.predict(xfit[:, np.newaxis])
        plt.plot(xfit, yfit, label = 'degree={}'.format(degree),linewidth=width)
    plt.legend(loc = 'best')


In [53]:
def all_features(dictionary):
    all_features = []
    for item in dictionary:
        all_features.extend(sp.audio_features(tracks = [item['track_id']]))
    return all_features


def add_features(all_features, dictionary):
    for dic in dictionary:
        for item in all_features:
            if dic['track_id'] == item['id']:
                dic['danceability'] = item['danceability']
                dic['energy'] = item['energy']
                dic['loudness'] = item['loudness']
                dic['key'] = item['key']
                dic['acousticness'] = item['acousticness']
                dic['valence'] = item['valence']
                dic['tempo'] = item['tempo']



In [54]:
def artist_features(features_for_tracks, dictionary):
    for dic in dictionary:
        for item in features_for_tracks:
            if item == None:
                pass
            else:
                if dic['track_id'] == item['id']:
                    dic['danceability'] = item['danceability']
                    dic['energy'] = item['energy']
                    dic['loudness'] = item['loudness']
                    dic['key'] = item['key']
                    dic['acousticness'] = item['acousticness']
                    dic['valence'] = item['valence']
                    dic['tempo'] = item['tempo']


In [55]:
def artist_tracks_dict(artist_name):
    list_tracks =[]
    for offset in range(0,500,50):
        x = sp.search(artist_name, 50, offset, type = 'track')
        temp = [track for track in x['tracks']['items']]
        list_tracks.extend(temp)
    top_track_dict = [{'album':item['album']['name'], 'album_id':item['album']['id'], 'album_release':item['album']['release_date'],'artist':item['artists'][0]['name'],'track_name':item['name'],'track_id':item['id']} for item in list_tracks]
    for piece in list_tracks:
        for section in top_track_dict:
            if piece['id'] == section['track_id']:
                if len(piece['artists']) > 1:
                    section['feature'] = piece['artists'][1]['name']
                else:
                    section['feature'] = 'No Feature'
    example = top_track_dict
    feat = all_features(example)
    artist_features(feat,example)
    return top_track_dict

In [52]:
sp.audio_features(tracks = ['7D8aQaRzoi9Qzz5yerVK5b'])

[{'danceability': 0.888,
  'energy': 0.586,
  'key': 0,
  'loudness': -6.745,
  'mode': 1,
  'speechiness': 0.0604,
  'acousticness': 0.00597,
  'instrumentalness': 0.0365,
  'liveness': 0.153,
  'valence': 0.401,
  'tempo': 150.051,
  'type': 'audio_features',
  'id': '7D8aQaRzoi9Qzz5yerVK5b',
  'uri': 'spotify:track:7D8aQaRzoi9Qzz5yerVK5b',
  'track_href': 'https://api.spotify.com/v1/tracks/7D8aQaRzoi9Qzz5yerVK5b',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7D8aQaRzoi9Qzz5yerVK5b',
  'duration_ms': 158757,
  'time_signature': 4}]

In [11]:
token = call_api('mrethana', 'user-top-read')
sp = spotipy.Spotify(auth=token)

In [56]:
artist = artist_tracks_dict('tinashe')

In [41]:
all_features = ['energy', 'valence', 'tempo','danceability','acousticness','key']

In [57]:
a_df = pd.DataFrame(artist)
a_df = a_df.query("artist == 'Tinashe'")

In [59]:
a_df.to_csv('artist.csv')

In [60]:
ldf = pd.read_csv('artist.csv', index_col = 0)

In [61]:
df = pd.read_csv('mrethana.csv', index_col = 0)

In [62]:
def relevant_features(dataframe):
    all_features = ['energy', 'valence', 'tempo','danceability','acousticness','key']
    rs = []
    for feature in all_features:
        yt,max_lambda =boxcox(dataframe.percentile +.01)
        xt,max_lambda=boxcox(dataframe[feature] + .01)
        r = regression(yt,xt)
        rs.append((r,feature))
    return [tuple[1] for tuple in sorted(rs, reverse = True)[0:3]]

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
features = relevant_features(df)

In [65]:
features

['valence', 'energy', 'key']

In [66]:
ldf

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence
1,0.005970,Throw A Fit,2kGxRa3ut2AMvo1fdoUeuK,2018-07-27,Tinashe,0.888,0.58600,No Feature,0,-6.745,150.051,7D8aQaRzoi9Qzz5yerVK5b,Throw A Fit,0.4010
2,0.150000,Aquarius,6zXUDBGLbrB9Kgkw2Y3F7L,2014-10-01,Tinashe,0.742,0.59500,ScHoolboy Q,9,-7.510,101.013,3jVtllWS5CFFWLQng8sKsr,2 On (feat. ScHoolboy Q),0.4310
3,0.571000,Like I Used To,3agSXLYruoxkcf3U6zPTN4,2018-07-13,Tinashe,0.717,0.59900,No Feature,11,-6.156,159.780,3fMKnwiByu9yfeD5ISn9Et,Like I Used To,0.3850
4,0.196000,Joyride,0u3rjsCgagcSxHoRtXMKQo,2018-04-13,Tinashe,0.748,0.65800,Offset,7,-4.656,143.954,3doVAxeH3SgWtlRMqR4eLi,No Drama,0.4480
7,0.678000,Joyride,0u3rjsCgagcSxHoRtXMKQo,2018-04-13,Tinashe,0.566,0.66900,Future,5,-5.772,102.582,17uivGWu6kfAvH6UgR7j4x,Faded Love,0.6980
8,0.298000,Joyride,0u3rjsCgagcSxHoRtXMKQo,2018-04-13,Tinashe,0.767,0.66900,Ty Dolla $ign,1,-4.508,99.933,6XddLa3bdlJJQC6sImufdH,Me So Bad,0.7200
11,0.386000,Aquarius,6zXUDBGLbrB9Kgkw2Y3F7L,2014-10-01,Tinashe,0.798,0.53200,No Feature,7,-4.329,96.992,1kXmApTm4wonqscYBoLwxj,All Hands on Deck,0.3720
12,0.552000,Company,734QluBStyfeKmcRgQUDO7,2016-09-16,Tinashe,0.897,0.49800,No Feature,9,-5.000,119.991,1608cOZINzEjaREvqEaDBd,Company,0.9600
14,0.493000,All Hands On Deck REMIX,4lP8vfD5YomYnhEvavof6R,2015-02-24,Tinashe,0.807,0.53000,Iggy Azalea,7,-4.236,97.007,5rX9fqEGsHnQeEEsc1Cc3J,All Hands On Deck REMIX,0.4290
15,0.027200,Player,68spG6Jww5Ya5tp060NNyl,2015-10-02,Tinashe,0.721,0.69000,Chris Brown,4,-5.930,106.042,6JdMhQybpCiM754NZDw2pm,Player,0.2920


In [67]:
ldf.key = ldf.key/11
df.key = df.key/11
cluster_df = df

In [68]:
X = cluster_df[['valence','energy','key']]
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
cluster_df['cluster'] = y_kmeans
print(len(y_kmeans[(y_kmeans == 0)]))
print(len(y_kmeans[(y_kmeans == 1)]))
print(len(y_kmeans[(y_kmeans == 2)]))
print(len(y_kmeans[(y_kmeans == 3)]))
print(len(y_kmeans[(y_kmeans == 4)]))
print(kmeans.cluster_centers_)
centers = kmeans.cluster_centers_

74
60
103
57
0
[[0.65212162 0.62595946 0.22481572]
 [0.285815   0.5448     0.11666667]
 [0.33453495 0.55931068 0.69549868]
 [0.69873684 0.72215789 0.79904306]]


In [70]:
list(centers)[0]

array([0.65212162, 0.62595946, 0.22481572])

In [71]:
x_cent = [i[0] for i in centers]
y_cent = [i[1] for i in centers]
z_cent = [i[2] for i in centers]

In [277]:
from plotly import __version__
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
print (__version__) # requires version >= 1.9.0

#Always run this the command before at the start of notebook
init_notebook_mode(connected=True)
import plotly.graph_objs as go

trace1 = go.Scatter3d(
    z=cluster_df.valence,
    x=cluster_df.energy,
    y=cluster_df.key,
    mode='markers',
    marker=dict(
        size=8,
        color=y_kmeans,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))


trace2 = go.Scatter3d(
    x=centers[:, 0],
    y=centers[:, 1],
    z=centers[:, 2],
    mode='markers',
    marker=dict(
        size=8,
        color='black',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.3
    )
)

data = [trace1,trace2]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title='Valence'),
                    yaxis = dict(
                        title='Energy'),
                    zaxis = dict(
                        title='Key'),),
                    width=700,
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

3.0.0


In [72]:
profile = np.array([df[feat].mean() for feat in features])

In [73]:
profile2d = np.array([df[feat].mean() for feat in features[0:2]])

In [74]:
compare = ldf[[features[0],features[1],features[2]]].values
compare2d = ldf[[features[0],features[1]]].values

In [75]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [76]:
def all_similarities(user,artist):
    cos_list = []
    for item in artist:
        x = cos_sim(user,item)
        cos_list.append(x)
    ldf['cos_similarity'] = cos_list

In [77]:
all_similarities(profile,compare)

In [78]:
profile

array([0.47513946, 0.60469728, 0.47897341])

In [79]:
ldf[(ldf.cos_similarity == ldf.cos_similarity.max())]

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity
114,0.0112,Superlove,4FgKLcQHC4LxTWLL0R4bOt,2016-07-15,Tinashe,0.772,0.624,No Feature,0.454545,-5.024,135.01,1pRoJqsXXqEAzdDUOOakt1,Superlove,0.448,0.999057


In [80]:
def euclidean_distance(user, artist):   
    return np.sqrt(np.sum((user - artist) ** 2))

In [81]:
ldf.nlargest(5,'cos_similarity')

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity
114,0.0112,Superlove,4FgKLcQHC4LxTWLL0R4bOt,2016-07-15,Tinashe,0.772,0.624,No Feature,0.454545,-5.024,135.010,1pRoJqsXXqEAzdDUOOakt1,Superlove,0.448,0.999057
129,0.0037,Superlove (The Remixes),2xbXciaeKylZrFl26YglZV,2016-09-09,Tinashe,0.668,0.623,The Golden Pony,0.454545,-7.875,91.114,6YA0PjO8ZKUi3YqaHc7hiO,Superlove - The Golden Pony Remix,0.519,0.998635
21,0.0148,Superlove,6hXXlsQi7USKqmeg8ytALG,2016-07-15,Tinashe,0.774,0.635,No Feature,0.454545,-4.991,134.989,1uTXtV2r7DSRnMGKh3HxGi,Superlove,0.438,0.998286
25,0.0419,Nightride,4oUQWJAcEtTuhYFR9AGfaC,2016-11-04,Tinashe,0.709,0.538,No Feature,0.454545,-5.821,113.945,5mhIx9uFL53Jk8X8XfLG6F,Sacrifices,0.385,0.998285
73,0.1380,Flame (Steve James Remix),5wfevG65FDgfa2pwPC4B7H,2017-06-30,Tinashe,0.643,0.646,Steve James,0.454545,-7.155,88.866,4IhZN36ncIl66lToIQbqMI,Flame - Steve James Remix,0.531,0.998071


In [82]:
def all_distance(user,artist):
    e_dist = []
    for item in artist:
        x = euclidean_distance(user,item)
        e_dist.append(x)
    return e_dist

In [83]:
ldf['e_1'] = all_distance(centers[0],compare)
ldf['e_2'] = all_distance(centers[1],compare)
ldf['e_3'] = all_distance(centers[2],compare)
ldf['e_4'] = all_distance(centers[3],compare)
ldf['euclidean'] = ldf[["e_1", "e_2",'e_3', 'e_4']].min(axis=1)

In [84]:
ldf[(ldf.euclidean == ldf.euclidean.min())]

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity,e_1,e_2,e_3,e_4,euclidean
52,0.484,Nightride,4oUQWJAcEtTuhYFR9AGfaC,2016-11-04,Tinashe,0.63,0.515,No Feature,0.727273,-8.45,130.031,4EabiixKs1IXXRyFha7ODG,Soul Glitch,0.338,0.950037,0.602866,0.613556,0.054635,0.422133,0.054635


In [85]:
ldf = ldf.drop_duplicates('track_name')
rec = ldf.nsmallest(15,'euclidean')

In [86]:
rec

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity,e_1,e_2,e_3,e_4,euclidean
52,0.48400,Nightride,4oUQWJAcEtTuhYFR9AGfaC,2016-11-04,Tinashe,0.630,0.515,No Feature,0.727273,-8.450,130.031,4EabiixKs1IXXRyFha7ODG,Soul Glitch,0.338,0.950037,0.602866,0.613556,0.054635,0.422133,0.054635
70,0.05020,Nightride,4oUQWJAcEtTuhYFR9AGfaC,2016-11-04,Tinashe,0.567,0.551,No Feature,0.727273,-9.537,101.043,7nYw37hFYh1BufGA2HT0Ol,Touch Pass,0.279,0.941760,0.630319,0.610676,0.064520,0.458939,0.064520
68,0.00332,All Hands On Deck (Giraffage Remix),6tYwzPFm9rmqQYTLQwfQTC,2015-05-04,Tinashe,0.813,0.592,Giraffage,0.272727,-7.733,113.997,2fzly0v7ndY7NydVFBE2E0,All Hands On Deck - Giraffage Remix,0.606,0.963063,0.074672,0.359307,0.503485,0.550045,0.074672
11,0.38600,Aquarius,6zXUDBGLbrB9Kgkw2Y3F7L,2014-10-01,Tinashe,0.798,0.532,No Feature,0.636364,-4.329,96.992,1kXmApTm4wonqscYBoLwxj,All Hands on Deck,0.372,0.975300,0.506624,0.526950,0.075143,0.411560,0.075143
96,0.48000,All Hands On Deck Remix,19Ca6MzHJkMzh6A630KK06,2015-02-24,Tinashe,0.808,0.526,Iggy Azalea,0.636364,-4.222,96.996,5LrlJ9f0MGTEMGXd2ixapd,All Hands On Deck Remix,0.392,0.977166,0.497018,0.530767,0.088931,0.398786,0.088931
27,0.05170,Joyride,0u3rjsCgagcSxHoRtXMKQo,2018-04-13,Tinashe,0.663,0.623,No Feature,0.727273,-5.935,124.990,5tnNHmykSmF7Nam6aenBVf,No Contest,0.277,0.948523,0.627047,0.615656,0.091522,0.439141,0.091522
72,0.44900,Aquarius,6zXUDBGLbrB9Kgkw2Y3F7L,2014-10-01,Tinashe,0.610,0.751,No Feature,0.727273,-6.784,83.030,4MlS9uLdQosggVJe91GJme,Far Side Of The Moon,0.768,0.993287,0.530590,0.804898,0.475022,0.103828,0.103828
14,0.49300,All Hands On Deck REMIX,4lP8vfD5YomYnhEvavof6R,2015-02-24,Tinashe,0.807,0.530,Iggy Azalea,0.636364,-4.236,97.007,5rX9fqEGsHnQeEEsc1Cc3J,All Hands On Deck REMIX,0.429,0.981203,0.477874,0.539264,0.115238,0.368981,0.115238
32,0.17800,Joyride,0u3rjsCgagcSxHoRtXMKQo,2018-04-13,Tinashe,0.694,0.495,No Feature,0.090909,-4.149,133.936,74RjqevhxutV5yZdE8bFX3,He Don't Want It,0.422,0.913161,0.296711,0.147275,0.614259,0.793497,0.147275
135,0.05280,Superlove (The Remixes),2xbXciaeKylZrFl26YglZV,2016-09-09,Tinashe,0.644,0.661,Frank Pole,0.727273,-4.654,125.974,2pKiDIuMiCyuKWbaYWR09V,Superlove - Frank Pole Remix,0.229,0.937229,0.657817,0.624156,0.149960,0.479107,0.149960


In [293]:
trace1 = go.Scatter3d(
    x=x_cent,
    y=y_cent,
    z=z_cent,
    mode='markers',
    marker=dict(
        size=18,
        color='blue',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=ldf[features[0]],
    y=ldf[features[1]],
    z=ldf[features[2]],
    mode='markers',
    marker=dict(
        size=12,
        color='red',                # set color to an array/list of desired values
        colorscale='Earth',   # choose a colorscale
        opacity=0.4
    ),text= ldf['track_name']
)


data = [trace1,trace2]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title=features[0]),
                    yaxis = dict(
                        title=features[1]),
                    zaxis = dict(
                        title=features[2]),),
                    width=700,
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='plot from API (17)')



In [ ]:
def call_api(username, scope):
    token = util.prompt_for_user_token(username,scope,client_id='4849430a2b114eae9d8eaf2c1f14f612',client_secret='11b854b009344409861112df56d9cf44',redirect_uri='http://127.0.0.1:8080/callback/q') 
    return token

id_add = list(rec['track_id'].to_dict().values()) #get track ID's of all recs

token2 = call_api('mrethana', 'playlist-modify-public') #create new api call with user and API scope
sp2 = spotipy.Spotify(auth=token2)

return_to_dict = rec['artist'].to_dict() #get dictionary of recs
artist_name = list(return_to_dict.values())[0] #get name of artist for playlist 

sp2.user_playlist_create('mrethana',artist_name, True) #create new playlist for user with title of artist name
newest_id = sp2.user_playlists('mrethana')['items'][0]['id'] #get ID of playlist just created

sp2.user_playlist_add_tracks('mrethana', newest_id, id_add, None) #populate playlist with all recs 


In [ ]:
def call_api(username, scope):
    token = util.prompt_for_user_token(username,scope,client_id='4849430a2b114eae9d8eaf2c1f14f612',client_secret='11b854b009344409861112df56d9cf44',redirect_uri='http://127.0.0.1:8080/callback/q') 
    return token

In [87]:
id_add = list(rec['track_id'].to_dict().values())

In [88]:
token2 = call_api('mrethana', 'playlist-modify-public')
sp2 = spotipy.Spotify(auth=token2)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opening https://accounts.spotify.com/authorize?client_id=4849430a2b114eae9d8eaf2c1f14f612&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8080%2Fcallback%2Fq&scope=playlist-modify-public in your browser


Enter the URL you were redirected to: http://127.0.0.1:8080/callback/q?code=AQCzA6nlfMmtPfu89jkmskfGjf599P9uMbcqAJ42r6O39StXOCQhMwdPqk-isFzUtdEUey9WNH8OiL3slFxxRo5sgi-pBptW170_GfH6OMNbQpJe5ziZGibTwD4SgFqFqhWDMEpl18SZmw00r437VfFlfk8AJiWZzqdc1sBDfQwSteKh3oeQiIwfUYR_kF9y9Lw49DysvYl_muJigUZ8oMADamD4ooEFe5n95m3Atzg




In [89]:
return_to_dict = rec['artist'].to_dict()
artist_name = list(return_to_dict.values())[0]
artist_name

'Tinashe'

In [90]:
sp2.user_playlist_create('mrethana',artist_name, True)

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5hMgEQXenDHtY7JXE9DqQB'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/5hMgEQXenDHtY7JXE9DqQB',
 'id': '5hMgEQXenDHtY7JXE9DqQB',
 'images': [],
 'name': 'Tinashe',
 'owner': {'display_name': 'mrethana',
  'external_urls': {'spotify': 'https://open.spotify.com/user/mrethana'},
  'href': 'https://api.spotify.com/v1/users/mrethana',
  'id': 'mrethana',
  'type': 'user',
  'uri': 'spotify:user:mrethana'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSwwYTMyMjFhMDVjM2ZjYzgzZDMwODEzMzBjMmMzZGNmMjE3NjUwODA4',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/5hMgEQXenDHtY7JXE9DqQB/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:user:mrethana:playlist:5hMgEQXenDHtY7JXE9DqQB'}

In [91]:
newest_id = sp2.user_playlists('mrethana')['items'][0]['id']

In [92]:
sp2.user_playlist_add_tracks('mrethana', newest_id, id_add, None)

{'snapshot_id': 'Miw3MzU4Njk3ZGNmOGIzMDAxZDQ4MjM0MDM4ZDNlOWQ1MzE3MmI2MTUx'}